In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Importar o ficheiro
df = pd.read_csv('C:\\Users\\guilh\\Desktop\\AASE\\Projeto\\csv\\train.csv', sep=";", header=0)

In [ ]:
                        #BRAND
#Como é possível verificar no gráfico de barras, a maior queda de valores é entre chevrolet para porshe. Portanto, vamos considerar os valores "Ford", "BMW", "Mercedes" e "Chevorlet", os outros valores vão ser populados com "Outro"
allowed_brands = ['Ford', 'Mercedes-Benz', 'BMW', 'Chevrolet']
df['brand'] = df['brand'].apply(lambda x: x if x in allowed_brands else 'Other')

In [ ]:
                        #MODEL
#Como vimos na matriz da correlação, a marca não interfere no preço do automóvel, pelo que pode ser removida.
#drop the model column
df.drop('model', axis = 1, inplace = True)

In [ ]:
                        #MODEL_YEAR
#Dividimos as datas tipo-  2000-2005 | 2005-2010 ... ?????
                 

In [ ]:
                        #MILAGE
#Alterar o tipo da coluna para numero e converter de milhas para kilometros, para melhor compreensão.

# Alterar a coluna milage para float e depois converter para km
df['milage'] = df['milage'].str.split(" ", expand = True)[0].str.replace(',', '').astype(float)


# converter milhas para kilometros
mile_to_km = 0.621371
df['kilometers'] = df['milage'] / mile_to_km

# Arredondar os valores em 2 casas decimais
df['kilometers'] = df['kilometers'].round(0).astype(int)


In [ ]:
                        #FUEL TYPE

# Juntar os hibridos todos, 
df.loc[df['fuel_type'].str.contains('Hybrid', na=False), 'fuel_type'] = 'Hybrid'
df.loc[df['fuel_type'].str.contains('–', na=False), 'fuel_type'] = 'No data'
df.loc[df['fuel_type'].str.strip() == '', 'fuel_type'] = 'No data'
df.loc[df['fuel_type'] == 'not supported', 'fuel_type'] = 'Hydrogen' # (só existe um carro com este valor: Toyota -Mirai Limited, movido a hidrogénio)
# Todos os valores que estão nulos são eletricos
df['fuel_type'].fillna('Eletric', inplace=True)


In [ ]:
                    #ENGINE

#Separar os valores do motor em "Horsepower", "Cilinders" and "Litres"
split_engine = df['engine'].str.split(' ',expand = True)

split_engine.loc[split_engine[0].str.contains('HP'), 'horsepower'] = split_engine.loc[split_engine[0].str.contains('HP'), 0].str.replace('HP', "").astype(float)
split_engine.loc[(split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 'litres'] = split_engine.loc[(split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 1].str.replace('L', "").astype(float)
split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == 'Cylinder'), 'cylinders'] = split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == 'Cylinder'), 2].str.replace('V', "").astype(int)
split_engine.loc[split_engine[0].str.contains('HP') & (split_engine[3] == '6'), 'cylinders'] = 6

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('L')), 'litres'] = pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('L')), 0].str.replace('L', ''), errors = 'coerce')
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[0].str.contains('V'))][0].str.replace('V', ''), errors = 'coerce'))


split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 'litres'] = pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('L')), 1].str.replace('L', ''), errors = 'coerce')
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('I')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('I'))][1].str.replace('I', '')))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('H')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('H'))][1].str.replace('H', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('W')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('W'))][1].str.replace('W', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('V')), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].str.contains('V'))][1].str.replace('V', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].isin([str(x) for x in list(range(20))])), 'cylinders'] = split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[1].isin([str(x) for x in list(range(20))]))][1].astype(float)

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('V')) & (split_engine['cylinders'].isnull())][2].str.replace('V', '')))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('I')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('I')) & (split_engine['cylinders'].isnull())][2].str.replace('I', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull()), 'litres'] = split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[2].str.contains('L')) & (split_engine['cylinders'].isnull())][2].str.replace('L', '').astype(float)


split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[3].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[3].str.contains('V')) & (split_engine['cylinders'].isnull())][3].str.replace('V', ''), errors = 'coerce'))

split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('I')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('I')) & (split_engine['cylinders'].isnull())][4].str.replace('I', ''), errors = 'coerce'))
split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('V')) & (split_engine['cylinders'].isnull()), 'cylinders'] = abs(pd.to_numeric(split_engine.loc[(~split_engine[0].str.contains('HP')) & (split_engine[4].str.contains('V')) & (split_engine['cylinders'].isnull())][4].str.replace('V', ''), errors = 'coerce'))

split_engine.loc[(split_engine[1] == 'Liter') & (split_engine['litres'].isnull()), 'litres'] = split_engine.loc[(split_engine[1] == 'Liter') & (split_engine['litres'].isnull())][0].astype(float)

df = pd.concat([df, split_engine[['horsepower', 'litres', 'cylinders']]], axis = 1)



df[['engine', 'horsepower', 'litres', 'cylinders']].head(100)

In [ ]:
                #ENGINE
#substituimos os valores nulos pela media dos valores
df['horsepower'].fillna(df['horsepower'].mean(), inplace = True)
df['litres'].fillna(df['litres'].mean(), inplace = True)
df['cylinders'].fillna(df['cylinders'].mean(), inplace = True)


In [ ]:
# TRANSMISSION
# Vamos manter apenas "Automatic", "Manual", "DCT", "No data"
df['transmission'] = df['transmission'].str.lower()

df.loc[df['transmission'].str.contains('automatic|a/t|at|auto|cvt'), 'transmission'] = 'Automatic'
df.loc[df['transmission'].str.contains('m/t|manual|mt'), 'transmission'] = 'Manual'
df.loc[df['transmission'].str.contains('dual shift mode'), 'transmission'] = 'DCT'
df.loc[df['transmission'].str.contains('–|2|f|7-speed|variable'), 'transmission'] = 'No data'

# Preencher valores nulos com 'No data'
df['transmission'].fillna('No data', inplace=True)


In [ ]:
                #EXT_COLOR


# Substituir 'White' por 'White'
df.loc[df['int_col'].str.contains('White', case=False), 'int_col'] = 'White'

# Substituir 'black' por 'black'
df.loc[df['int_col'].str.contains('Black', case=False), 'int_col'] = 'Black'

# Substituir todos os valores que não são "white" ou "black" por "other"
df.loc[~df['ext_col'].isin(['White', 'Black']), 'ext_col'] = 'Other'

In [ ]:
                #INT_COLOR


# Substituir os valores baseados em palavras-chave

# Substituir 'beige' por 'beige'
df.loc[df['int_col'].str.contains('Beige', case=False), 'int_col'] = 'Beige'

# Substituir 'black' por 'black'
df.loc[df['int_col'].str.contains('Black', case=False), 'int_col'] = 'Black'

# Substituir 'gray' por 'gray'
df.loc[df['int_col'].str.contains('Gray', case=False), 'int_col'] = 'Gray'

# Substituir todos os valores que não são "white" ou "black" por "other"

df.loc[~df['int_col'].isin(['Beige', 'Black', "Gray"]), 'int_col'] = 'Other'

In [ ]:
                #CLEAN_TITLE
# Substituir os valores ausentes na coluna 'clean_title' por "No data"
df['clean_title'].fillna("No data", inplace=True)


In [ ]:
                #ACCIDENT
# Substituir os valores ausentes na coluna 'clean_title' por "No data"
df['accident'].fillna("No data", inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Seleciona apenas as colunas não numéricas
categorical_columns = df.select_dtypes(include='object').columns

# Inicializa o LabelEncoder
label_encoder = LabelEncoder()

# Mapeia as categorias para números
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Calcule a matriz de correlação de Spearman
correlation_matrix = df.corr(method='spearman')

# Visualize a matriz de correlação usando um heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlação de Spearman entre Variáveis (Categóricas Mapeadas para Números)')
plt.show()


In [ ]:
#------------------------------REMOVER TODAS AS COLUNAS QUE A CORRELAÇÃO COM PREÇO SEJA INFERIOR A 0.5 -----------
colunas_para_eliminar = ['cylinders', 'litres', "clean_title", "accident", "int_col", "ext_col", "transmission", "engine", "fuel_type", "milage", "brand" ]
# Eliminar as colunas especificadas
df = df.drop(colunas_para_eliminar, axis=1)

In [ ]:
# MODELING

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from sklearn.metrics import (
        make_scorer,
        confusion_matrix
)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor # decision trees for classification
from sklearn.naive_bayes import GaussianNB # naive bayes for classification
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC # support vector machines for classification

In [ ]:
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn+fp)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
METRICS = {
    "MAE": make_scorer(mean_absolute_error),
    "MSE": make_scorer(mean_squared_error),
    "RMSE": make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred))),
    "R2": make_scorer(r2_score)
}

In [ ]:
df = df.dropna()

In [ ]:
X, y = df.drop("price", axis=1), df["price"]

In [ ]:
### Árvores de Decisão
dt = DecisionTreeRegressor(max_depth=3, random_state=1234)
dt.fit(X, y)  # Treinando o modelo
splitter = StratifiedKFold(10, random_state=1234, shuffle=True)
scores = cross_validate(dt, X, y, cv=splitter, scoring=METRICS)
dt_scores = pd.DataFrame(scores)
pd.DataFrame(dt_scores.mean()).T

In [ ]:
### NAIVE BAYNES
nb = GaussianNB()
scores_nb = cross_validate(nb, X, y, cv=splitter, scoring=METRICS)
nb.fit(X, y)  # Treinando o modelo
nb_scores = pd.DataFrame(scores_nb)
pd.DataFrame(nb_scores.mean()).T

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=1234)

# Perform cross-validation with the Random Forest Regressor
scores_rf = cross_validate(rf, X, y, cv=splitter, scoring=METRICS)

# Display the results
rf_scores = pd.DataFrame(scores_rf)
rf.fit(X, y)  # Treinando o modelo
pd.DataFrame(rf_scores.mean()).T


In [ ]:
# VETOR MACHINE
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create an SVR model with a pipeline for scaling
svr = make_pipeline(StandardScaler(), SVR())

# Perform cross-validation with the SVR
scores_svr = cross_validate(svr, X, y, cv=splitter, scoring=METRICS)

# Display the results
svr.fit(X, y)  # Treinando o modelo
svr_scores = pd.DataFrame(scores_svr)
pd.DataFrame(svr_scores.mean()).T


In [ ]:
#REDES NEURONAIS
from sklearn.neural_network import MLPRegressor

# Create an MLP Regressor
mlp = MLPRegressor(random_state=1234)

# Perform cross-validation with the MLP Regressor
scores_mlp = cross_validate(mlp, X, y, cv=splitter, scoring=METRICS)

# Display the results
mlp_scores = pd.DataFrame(scores_mlp)
mlp.fit(X, y)  # Treinando o modelo
pd.DataFrame(mlp_scores.mean()).T


In [ ]:
# Modelos e nomes para os gráficos
models = [dt, nb, rf, svr, mlp]
names = ['Decision Tree', 'Naive Bayes', 'Random Forest', 'SVR', 'Neural Network']

# Criando os gráficos de dispersão
fig, axes = plt.subplots(5, 1, figsize=(20, 25))
plt.subplots_adjust(hspace=0.2)

for e, ax in enumerate(axes):
    model = models[e]

    y_pred = model.predict(X)  # Agora, o modelo está treinado

    sns.scatterplot(x=y.values.ravel(), y=y_pred.ravel(), ax=ax)
    ax.set_title(names[e])

plt.show()